In [ ]:
#|hide
#|eval: false
! [ -e /content ] && pip install -Uqq fastrl['dev'] pyvirtualdisplay && \
                     apt-get install -y xvfb python-opengl > /dev/null 2>&1 
# NOTE: IF YOU SEE VERSION ERRORS, IT IS SAFE TO IGNORE THEM. COLAB IS BEHIND IN SOME OF THE PACKAGE VERSIONS

In [ ]:
#|hide
#|eval: false
from fastcore.imports import in_colab
# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev.showdoc import *
    from nbdev.imports import *
    if not os.environ.get("IN_TEST", None):
        assert IN_NOTEBOOK
        assert not IN_COLAB
        assert IN_IPYTHON
else:
    # Virutual display is needed for colab
    from pyvirtualdisplay import Display
    display = Display(visible=0, size=(400, 300))
    display.start()

In [ ]:

# Python native modules

# Third party libs
import torch
import numpy as np
# Local modules

# Conjugation
> Notes and functions illistrated by [Shewchuk, 1994](https://www.cs.cmu.edu/~quake-papers/painless-conjugate-gradient.pdf) and will
be referenced through this notebook.

The [Shewchuk, 1994](https://www.cs.cmu.edu/~quake-papers/painless-conjugate-gradient.pdf) bases 
all the example problems on sample problem 4...

In [ ]:
A = torch.tensor(
    [[3.,2.],[2.,6.]]
)
b = torch.tensor([[2.],[-8.]])
c = 0

In [ ]:
# x minimizes the following function `f`
x = torch.tensor([[2.],[-2.]])

## Basics
We define a quadratic function whose minimum value output is -10. The challenge is
to pretend we don't know this, and automatically figure out what value of `x` is needed
to figure this out.

In [ ]:
def f(x): return (1/2) * x.T @ A @ x - b.T @ x + c

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
from plotly.subplots import make_subplots
pio.renderers.default = "plotly_mimetype+notebook_connected"

In [ ]:
xx = torch.tensor(np.array([x for x in np.ndindex(20,20)])).float()-10

def plot3d(xx,f):
    return px.scatter_3d(
        x=xx[:,0],
        y=xx[:,1],
        z = [f(x.reshape(-1,1)).numpy()[0][0] for x in xx]
    )
plot3d(xx,f)

In [ ]:
def f_prime(x):
    return (1/2) * A.T @ x + (1/2) * A @ x - b

Using `f_prime` above, we can gradients as vectors. For example given the location...

In [ ]:
xx = torch.tensor(np.array([x for x in np.ndindex(20,20)])).float()-15
xx[0]

We get the gradient...

In [ ]:
f_prime(xx[0].reshape(-1,1))

Which is extremely large relative to the original point. We can make a guess that
this is pretty far away from the solution. We can get the magnitude via..

In [ ]:
def get_magnitude(derivative_x,x):
    return torch.linalg.norm(x-derivative_x).numpy()

In [ ]:
print(
'Magnitude: '+str(get_magnitude(f_prime(xx[0].reshape(-1,1)),xx[0])),
'\nGadient Vector: '+str(f_prime(xx[0].reshape(-1,1))),
'\nOriginal point location: '+str(xx[0]))

Like we said earlier, using the `f_prime` function, finding which points are 
farthur from the solution is pretty easy. Remember that solution is `[2,-2]`, 
so we would expect points closer to here to have a smaller magnitude...

In [ ]:
print(
'Magnitude: '+str(get_magnitude(f_prime(xx[-1].reshape(-1,1)),xx[-1])),
'\nGadient Vector: '+str(f_prime(xx[-1].reshape(-1,1))),
'\nOriginal point location: '+str(xx[-1]))

When we plot the magnitudes of the gradients we get a nice slope where the solution `[2,-2]` is
the lowest value!

In [ ]:
def plot3d(xx,f):
    return px.scatter_3d(
        x=xx[:,0],
        y=xx[:,1],
        z = [get_magnitude(f_prime(x.reshape(-1,1)),x) for x in xx]
    )
plot3d(xx,f_prime)

Although the above equations give off the impression that this is only useful for a "path finding" 
kind of scenario, this actually scales out into more general parameter optimization. 

The solution `[2,-2]` could instead be many dimensions `[2,-2,1,6,3,...]` in which
case the algorithm of choice becomes more important since the solution search 
could be much harder.

## Automatically finding the solution: Method of Steepest Descent

In [ ]:
xx = torch.tensor(np.array([x for x in np.ndindex(100,100)])).float()-50

Note: [Shewchuk, 1994](https://www.cs.cmu.edu/~quake-papers/painless-conjugate-gradient.pdf) defines `error` as $error\, e_{(i)} = x_{(i)} - x$ which is used
to indicate the distance from the solution... However if we already know the solution, then why bother with anything discuessed in the paper? Isn't the whole 
point in all this that we don't know the solution?  Or maybe the $x$ in this scenario is the "ideal" solution,
but we don't know whether it is actually possible.

In machine learning we techinically have the solution already known, however we don't actually want to immediately optimize to it. In other words,
there can be many $x$s and we want to optimize to all of them in a general way. In other-other words, we want to get close-enough.

In [ ]:
def error(x,x_i):
    return x_i - x

We also have residual $r_i$ which can have multiple definitions:

$r_i = b - Ax_i$ is how far we are from the correct value of $b$

$r_i = -Ae_i$ is the error transformed by $A$ into the same space $b$

$r_i = -f'(x_i)$ is the direction of steepest descent 

However `Shewchuk, 1994` notes that only the last definition applies for non-lienar problems. 
This makes sense since the first definition can be found by setting $r_i$ to zero,
the second would require knowledge of `e` and `x`. 

With that considered, we define the residual as:


In [ ]:
def non_linear_residual(x_i):
    return -f_prime(x_i)

We can define a basic stepping function as $x_1 = x_0 + \alpha r_0$

Where $\alpha$ indicates how big of a step we should take

In [ ]:
def sd_step(x_i,r_func,alpha):
    return x_i + alpha * r_func(x_i)

In this step example we have `xx[0]`

In [ ]:
xx[0].reshape(-1,1)

Since this is `[-50,-50]`, it is very far from the solution `[-2,2]`, so the residual will be 
pretty massive, and likely can over shoot the solution.

In [ ]:
sd_step(xx[0].reshape(-1,1),non_linear_residual,0.01)

We define `naive_line_search` just take a step for `n_iterations` with a struct 
alpha. One obvious change would be to make `alpha` more dynamic such as start big,
but then get small if we think we are getting to a solution...

In [ ]:
def naive_line_search(x,n_iterations,alpha=0.01):
    steps = []
    for i in range(n_iterations):
        steps.append(x)
        x = sd_step(x,non_linear_residual,alpha)
    return steps

We find that the line search eventually optimizes to the solution!

In [ ]:
naive_line_search(xx[0].reshape(-1,1),100,0.1)[::10]

In [ ]:
def plot3d(xx,f,line_search_func):
    fig = go.Figure()
    mags = [get_magnitude(f_prime(x.reshape(-1,1)),x) for x in xx]
    fig.add_trace(
        go.Scatter3d(
            x=xx[:,0],
            y=xx[:,1],
            z = mags,
            mode='markers',
            name="Gradient Magnitudes"
        )
    )
    steps = torch.vstack([o.reshape(1,-1) for o in line_search_func(xx[40].reshape(-1,1),100,0.1)])
    # print(steps)
    fig.add_trace(
        go.Scatter3d(
            x=steps[:,0],
            y=steps[:,1],
            z = [get_magnitude(f_prime(x.reshape(-1,1)),x)+100 for x in steps],
            mode='lines+markers',
            name="Steps taken"
        )
    )
    return fig
plot3d(xx,f_prime,naive_line_search)

Success! We see that the final red dot is on the solution `[-2,2]` given an alpha of `0.1` and 100 steps. 
However how did we choose `alpha`? There might be a more automatic way of picking this.

[Shewchuk, 1994](https://www.cs.cmu.edu/~quake-papers/painless-conjugate-gradient.pdf) pg 6 choice of alpha should be such that $\frac{d}{d\alpha}f(x_1) = 0$.
Which if we remember that $x_1 = x_0 + \alpha r_0$, then $\frac{d}{d\alpha}f(x_1) = 0$ turns into $f'(x_1)^{T}r_0 = 0$ implying that $f'(x_1)^{T}$ and $r_0$ are orthogonal to each other.

In the end, the final equation is $\alpha = \frac{r^{T}_{0}r_{0}}{r^{T}_{0}A r_{0}}$

Note that $r^{T}_{0}A r_{0}$ and/or $r^{T}_{0}r_{0}$ will be zero if perfectly orthogonal. 

In [ ]:
def sd_step2(x_i,r,alpha):
    return x_i + alpha * r

def alpha_calc_line_search(x,n_iterations,alpha=0.01):
    steps = []

    for i in range(n_iterations):
        steps.append(x)
        r = non_linear_residual(x)
        # $\alpha = \frac{r^{T}_{0}r_{0}}{r^{T}_{0}A r_{0}}$
        alpha = (r.T@r) / (r.T @ (A@r))[0][0]
        x = sd_step2(x,r,alpha)
    return steps

plot3d(xx,f_prime,alpha_calc_line_search)

By simply changing how `alpha` is calculated, we end up taking orthogonal jumps between gradients resulting in 
much faster convergence, and removing the need for a togglable parameter.

## Automatically finding the solution: Jacobi Iteration
Note pg 11 [Shewchuk, 1994](https://www.cs.cmu.edu/~quake-papers/painless-conjugate-gradient.pdf) considers
Jacobi iteration more useful.

In this instance we take `A` and break it into 2 sub matricies: `E` and `D`.
Where `D` has the diagonal elements, while non-diagonal are zero, and 
`E` has the diagonal elements as zero, and non-diagonal are kept. 

Both `E` and `D` are the same shape as `A` and in fact `A` can be reconstructed from them
via $A=E+D$

The Jacobi Method is illistrated as:

$$
Ax = b
\\
Dx = -Ex + b
\\
x = -D^{-1}+D^{-1}b
\\
x = Bx + z
$$
where 
$$
B = -D^{-1}E
\\
z = D^{-1}b
$$

Which this considered, we get our x search space...

In [ ]:
xx = torch.tensor(np.array([x for x in np.ndindex(100,100)])).float()-50

We split `A` into `D` and `E`

In [ ]:
def calculate_d_e():
    D = A*torch.eye(A.shape[0])
    E = A.clone()
    non_zero = torch.eye(A.shape[0]).nonzero()
    E[non_zero[:,0],non_zero[:,1]] = 0
    return D,E
D,E = calculate_d_e()
A,D,E

Now we can create the iterative step process. This is illustrated as:
$$
x_{i+1} = Bx_{i}+z
$$

In [ ]:
def jacobi_step(x_i):
    B = -D.inverse()@E 
    z = D.inverse()@b 
    return B@x_i + z

In [ ]:
jacobi_step(xx[0].reshape(-1,1)),xx[0].reshape(-1,1)

In [ ]:
def jacobi_line_search(x,n_iterations):
    steps = []
    for i in range(n_iterations):
        steps.append(x)
        x = jacobi_step(x)
    return steps

So if we do a basic line search, after 20-25 iterations we are able to solve `Ax-b = 0`

In [ ]:
jacobi_line_search(xx[0].reshape(-1,1),25)[::5]

A couple iteresting notes from the author:

    splitting A differently — that is,
    by choosing a different  and  — we could have derived the Gauss-Seidel method, or the method of
    Successive Over-Relaxation (SOR). Our hope is that we have chosen a splitting for which has a small
    spectral radius. Here, I chose the Jacobi splitting arbitrarily for simplicity.

So there are other methods that use different strategies to splitting `A` that we could try.

In [ ]:
def plot3d(xx):
    fig = go.Figure()
    mags = [get_magnitude(f_prime(x.reshape(-1,1)),x) for x in xx]
    fig.add_trace(
        go.Scatter3d(
            x=xx[:,0],
            y=xx[:,1],
            z = mags,
            mode='markers',
            name="Gradient Magnitudes"
        )
    )
    steps = torch.vstack([o.reshape(1,-1) for o in jacobi_line_search(xx[40].reshape(-1,1),100)])
    # print(steps)
    fig.add_trace(
        go.Scatter3d(
            x=steps[:,0],
            y=steps[:,1],
            z = [get_magnitude(f_prime(x.reshape(-1,1)),x)+100 for x in steps],
            mode='lines+markers',
            name="Steps taken"
        )
    )
    return fig
plot3d(xx)

> Warning: On pg 12 it is noted that "Unfortunately,Jacobi does not converge for every A, or even for every positive-definite A"

### Eigen Vectors
The above method requires anylysis via eigen vectors to verify that we can even use
Jacobi effectively. `numpy` has `np.linalg.eig` and `torch` has a 
function with a similar name that can do this.

The main point in this section is that $\rho(B) < 1$ which is defined as the spectral radius. 
Greater than 1 means that the Jacobi optimization will not converge. 
If $\rho(B) < 1$, then we can gauge how quickly we will converge based on the 
eigenvector being used.

[Shewchuk, 1994](https://www.cs.cmu.edu/~quake-papers/painless-conjugate-gradient.pdf) uses an example of finding and using an eigenvectors / eigenvalues. 

[3Blue1Brown](https://www.youtube.com/watch?v=PFDu9oVAE-g&ab_channel=3Blue1Brown) does a good visual breakdown of what they are.

Given `A`:

In [ ]:
A

We will be finding eigenvectors `v` and eigenvalues $\lambda$

$Av = \lambda v = \lambda I v$

alternatively can be represented as:

$(\lambda I - A)v = 0$

[Shewchuk, 1994](https://www.cs.cmu.edu/~quake-papers/painless-conjugate-gradient.pdf) pg 12 also notes that 
"eigenvectors are nonzero, so $\lambda I - A$ must be singular then..."

$(\lambda I - A)v = 0$

$$
det 
\begin{pmatrix}
3 & 2 \\
2 & 6 
\end{pmatrix}
$$

Results in the 2 degree polynomial...

In [ ]:
p = np.polynomial.Polynomial([14,-9,1]);p

which if we factor out we get $(\lambda-7)(\lambda-2)$

In [ ]:
p.roots()

Which each of these are the eigenvalues, now we just need the eigenvectors...

In [ ]:
def calc_vector(mat,eigenvalue):
    "$\lambda I - A$ part of the $(\lambda I - A)v$"
    return eigenvalue * torch.eye(mat.shape[0]) - mat

In [ ]:
calc_vector(A,7)

We can find the eigen vectors be solving the equation below...

$$
(\lambda I - A)v = 

\begin{bmatrix}
4 & -2 \\
-2 & 1 
\end{bmatrix}

\begin{bmatrix}
v_1 \\
v_2 
\end{bmatrix}

= 4v_1 - 2v_2

= 0 
$$

In [ ]:
def calculate_d_e(mat):
    D = mat*torch.eye(mat.shape[0])
    E = mat.clone()
    non_zero = torch.eye(mat.shape[0]).nonzero()
    E[non_zero[:,0],non_zero[:,1]] = 0
    return D,E
D,E = calculate_d_e(A)
A,D,E

In [ ]:
def jacobi_step(x_i):
    B = -D.inverse()@E 
    z = D.inverse()@b 
    return B,B@x_i + z

In [ ]:
B,x_i = jacobi_step(xx[0]);B

In [ ]:
eigen_values,eigen_vectors = np.linalg.eig(B);eigen_values,eigen_vectors

`np.linalg.eig` normalizes the `eigen_vectors` to be unit length. We do a 
vvery simple un-normalize just so we can match the book. 

Some of the rational can be found [here](https://stackoverflow.com/a/47803336/4577212)

In [ ]:
eigen_vectors /= eigen_vectors[1,:]
eigen_vectors

Eigenvectors of `B` are $[\sqrt{2},1]^T$ and $[-\sqrt{2},1]^T$ where eigenvalues are respectively
$-\sqrt{2}/3$ and $\sqrt{2}/3$ 

In [ ]:
# We print here to show that the np.linalg.eig produces the same values as in the book
print('Eigen values: ',-np.sqrt(2)/3,' ',np.sqrt(2)/3)
print('Eigen vectors: ',[np.sqrt(2),1],' ',[-np.sqrt(2),1])

#### Now what?

Lets output the eigen vector components during a regular Jacobi optimization sequence.

In [ ]:
def jacobi_line_search(x,n_iterations):
    steps,errs = [],[]
    for i in range(n_iterations):
        old_x = x
        B,x = jacobi_step(x)
        err = torch.hstack((x,old_x)) # The book has x_i - x, however we need a vector.
        if err.sum()!=0:
            errs.append(err)
            steps.append(old_x)
    return steps,errs

In [ ]:
import plotly.express as px

In [ ]:
col_pal = px.colors.sequential.Rainbow
col_pal_iterator = itertools.cycle(col_pal)

def plot_arrow(fig,x1,y1,x2,y2,z1=1,z2=1,color=None, row=2, col=1):
    fig.add_trace(
        go.Scatter3d(
            x=[x1,x2],
            y=[y1,y2],
            z = [z1,z2],
            mode='lines',
            name=f'eig of:\n {x2.item()}\n{y2.item()}',
            showlegend=False,
            line=dict(color=color)
        ),
        row=row, col=col
    )

In [ ]:
def plot3d(xx):
    fig = make_subplots(rows=3, cols=1,
                        specs=[[{"type": "scene"}], [{"type": "scene"}], [{"type": "scene"}]])
    mags = [get_magnitude(f_prime(x.reshape(-1,1)),x) for x in xx]
    fig.add_trace(
        go.Scatter3d(
            x=xx[:,0],
            y=xx[:,1],
            z = mags,
            mode='markers',
            name="Gradient Magnitudes",
        ),
        row=1, col=1
    )
    step_errs = torch.vstack([torch.hstack((step.reshape(1,-1),err.reshape(1,-1))) for step,err in zip(*jacobi_line_search(xx[40].reshape(-1,1),100))])
    fig.add_trace(
        go.Scatter3d(
            x=step_errs[:,0],
            y=step_errs[:,1],
            z = [get_magnitude(f_prime(x.reshape(-1,1)),x)+100 for x in step_errs[:,:2]],
            mode='lines+markers',
            name="Steps taken",
        ),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter3d(
            x=step_errs[:,2],
            y=step_errs[:,3],
            z = [1 for x in step_errs],
            mode='lines+markers',
            name="Error / Change in X taken",
        ),
        row=2, col=1
    )
    for err in step_errs[:5]:
        # We are kind of feeding the wrong value into here. The error
        # is a locationless magnitude of x1,y1,x2,y2. Really we should be feeding
        # those into here, since we want to know the transformation matrix of those.
        eigv,eig_vec = torch.linalg.eig(
            torch.vstack(
                (err[2:4].reshape(1,-1),
                 err[2:4].reshape(1,-1))
        ))
        eigv,eig_vec = eigv.float(),eig_vec.float()
        # We calculate the actual scaled eigen vectors, then offset them to the 
        # primary vector
        e1 = err[2:4][0] + eig_vec[:,0] * eigv[0]
        e2 = err[2:4][1] + eig_vec[:,1] * eigv[1]

        color = next(col_pal_iterator)

        # First eigen vector
        plot_arrow(
            fig,
            x1=e1[0],  # arrows' head
            y1=e1[1],  # arrows' head
            x2=err[2:4][0],  # arrows' tail
            y2=err[2:4][1],  # arrows' tail
            color=color
        )
        # Second eigen vector
        plot_arrow(
            fig,
            x1=e2[0],  # arrows' head
            y1=e2[1],  # arrows' head
            x2=err[2:4][0],  # arrows' tail
            y2=err[2:4][1],  # arrows' tail
            color=color
        )
        ### Lets also plot the eigen vector components by themselves ###
        # First eigen vector
        plot_arrow(
            fig,
            x1=e1[0],  # arrows' head
            y1=e1[1],  # arrows' head
            x2=err[2:4][0],  # arrows' tail
            y2=err[2:4][1],  # arrows' tail
            color=color,
            row=3,col=1
        )
        # Second eigen vector
        plot_arrow(
            fig,
            x1=e2[0],  # arrows' head
            y1=e2[1],  # arrows' head
            x2=err[2:4][0],  # arrows' tail
            y2=err[2:4][1],  # arrows' tail
            color=color,
            row=3,col=1
        )

    fig.update_layout(
        autosize=False,
        width=1000,
        height=1800,
        margin=dict(
            l=50,
            r=50,
            b=100,
            t=100,
            pad=4
        ),
        # paper_bgcolor="LightSteelBlue",
    )
    return fig
plot3d(xx)

### General Convergence  6.2
pg 17


In [ ]:
def calculate_w(x1,x2):
    "Based on equation 25, pg 18"
    error = (x1 - x2).reshape(-1,2,1)

    eigv,eigvec = torch.linalg.eig(torch.hstack((error,error)).reshape(-1,2,2))
    eigv,eigvec = eigv.float(),eigvec.float()
    # So pg 15 has a var: \gamma_j ... I think that this is the same as 
    # error since error is basically the difference of components of x1 and x2
    e_i = (error * eigv.unsqueeze(-1)).sum(1)
    
    w_squared = 1 - (
        (((error[:,0]**2)*(eigv[:,0,None]**2) + (error[:,1]**2)*(eigv[:,1,None]**2))**2)
        /
        (
            ((error[:,0]**2)*(eigv[:,0,None]) + (error[:,1]**2)*(eigv[:,1,None])) 
            * ((error[:,0]**2)*(eigv[:,0,None]**3) + (error[:,1]**2)*(eigv[:,1,None]**3))
        )
    )
    return w_squared

In [ ]:
steps,err = jacobi_line_search(xx[40].reshape(-1,1),100)

In [ ]:
calculate_w(
    torch.hstack(steps[:-1]).T,
    torch.hstack(steps[1:]).T
)

In [ ]:
def plot3d(xx):
    fig = make_subplots(rows=1, cols=1,
                        specs=[[{"type": "scene"}]])
    steps,errors = jacobi_line_search(xx[40].reshape(-1,1),100)

    step_errs = torch.vstack([torch.hstack((step.reshape(1,-1),err.reshape(1,-1))) for step,err in zip(*(steps,errors))])


    ws = calculate_w(
        torch.hstack(steps[:-1]).T,
        torch.hstack(steps[1:]).T
    )

    ws = torch.vstack((ws[0],ws))
    ws[torch.isnan(ws)] = 0

    fig.add_trace(
        go.Scatter3d(
            x=step_errs[:,2],
            y=step_errs[:,3],
            z = [w.item() for w in ws],
            mode='lines+markers',
            name="Error / Change in X taken (adjusted by 100)",
        ),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter3d(
            x=step_errs[:,2],
            y=step_errs[:,3],
            z = [0.0000001 for x in step_errs],
            mode='lines+markers',
            name="Error / Change in X taken",
        ),
        row=1, col=1
    )

    fig.update_layout(
        autosize=False,
        width=1000,
        height=1000,
        margin=dict(
            l=50,
            r=50,
            b=100,
            t=100,
            pad=4
        ),
        # paper_bgcolor="LightSteelBlue",
    )
    return fig
plot3d(xx)

In [ ]:
#|hide
#|eval: false
from fastcore.imports import in_colab

# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev import nbdev_export
    nbdev_export()